In [ ]:
import os
import sys
from pathlib import Path
import pandas as pd

%matplotlib inline

In [ ]:
sys.path.append(r"../LUCinSA_helpers")
from rf import *

In [ ]:
'''
PARAMETERS: modify in notebook_params notebook, then run that notebook and this cell to update here
DO NOT modify this cell
'''

%store -r basic_config
print("Basic Parameters: \n time-series data is in (smooth_dir): {} \n"
      " modelling year is (filter_year param): {} (this is first year if season spans two years)"
      .format(basic_config['smooth_dir'], basic_config['filter_yr']))
%store -r classification_params

print("Classification_Params: \n" 
      " modelling mode is {} \n"
      " model_type = {} \n"
      " output files are saved to (model_dir): {} \n" 
      " shared input files are in (main_model_dir): {} \n"
      " sample_model = {} \n feature_model = {} \n model_name = {} \n"
      " the full sample pt file: {} \n"
      " the full sample dataframe with the feature model applied: {} \n"
      " the subset pt file based on the sample model: {} \n"
      " lc_class = {} \n ranhold = {} \n impmeth = {}"
      .format(classification_params['model_mode'],classification_params['model_type'],classification_params['model_dir'],
              classification_params['main_model_dir'],classification_params['sample_model'],classification_params['feature_model'],
              classification_params['model_name'],basic_config['ptfile'],classification_params['samp_pix_vars'],classification_params['samp_pts'],
              classification_params['lc_mod'],classification_params['ranhold'],classification_params['impmeth']))
    

#### Set new variables here for temp model testing: -- SKIP if keeping original model

In [ ]:
## Set new variables here for temp model testing:
feature_model = "Max"
## Sample model options currently: bal400mix1 | bal400mix2 | bal400mix3
sample_model = "bal400mix3"

## The following will set themselves based on the above variables:
classification_params['feature_model'] = feature_model
classification_params['sample_model'] = sample_model
classification_params['model_name'] = '{}_{}'.format(feature_model, sample_model)
classification_params['samp_pix_vars'] = '{}/ptsgdb_{}.csv'.format(classification_params['model_dir'],feature_model)
classification_params["samp_pts"] = '/home/downspout-cel/paraguay_lc/classification/RF/sample_dfs/{}.csv'.format(sample_model)
print('Now working with sample_model: {} \n New output model will be named: {}'
      .format(classification_params['sample_model'],classification_params['model_name']))

## Merge dataframes together

In [ ]:
lut=pd.read_csv('../Class_LUT.csv')
#print(lut.sort_values('LC_UNQ')[['LC_UNQ','USE_NAME','LC25','LC25_name']])

samp_pts = pd.read_csv(classification_params['samp_pts'])
print(samp_pts.columns.tolist())
#if mod_name == "base1000":
#    samp_pts.rename(columns = {"Unnamed: 0": 'OID_'}, inplace = True)

pix_vars = pd.read_csv(classification_params['samp_pix_vars'])
#print(pix_vars.columns.tolist())

pix_data = samp_pts.merge(pix_vars, left_on='OID_', right_on='OID_', how='inner')
print('sample breakdown by LC25 class:')
print(pix_data['LC25_name'].value_counts())

#pixdf = pix_data.merge(lut, left_on='Class', right_on='USE_NAME', how='left')

classification_params["pixdf"] = pix_data
#print(classification_params['pixdf'].columns.tolist())

## View the look up table
These are the different LC_models to group things in classification and to translate between numerical map categories and text labels

In [ ]:
lut=pd.read_csv('../Class_LUT.csv')
lut.drop(['Description'], axis=1, inplace=True)

print(lut.sort_values('LC_UNQ'))

## get sample dataframe:
 pixdf is the combination of the sample point file and variable stack for those points (pix_vars).
 This is created in notebooks 6a and 6b.

In [ ]:
"""print(classification_params['pixdf'])
pixdf = pd.read_csv(classification_params['pixdf'])
#pixdf = pixdf.dropna(inplace = True)
#print(pixdf)
#print(pixdf.isnull().any().any())
print('sample breakdown by {}:'.format(classification_params['lc_mod']))
label_col, new_lut = get_class_col(classification_params['lc_mod'], lut)
if '{}_name'.format(label_col) in pixdf.columns:
    print(pixdf['{}_name'.format(label_col)].value_counts())
else:
    pixdf2 = pixdf.merge(new_lut[['USE_NAME','{}'.format(label_col),'{}_name'.format(label_col)]], left_on='Class', right_on='USE_NAME', how='left')
    print(pixdf2['{}_name'.format(label_col)].value_counts())""";

## create rf model
this uses the multiclass RandomForestClassifier method from sklearn.ensemble (code is in ../LUCinSA_helpers/rf.py)

To use a different classification model, change 'lc_mod' in the parameters and rerun
current models = ('All' | 'trans_cats" | 'crop_nocrop' | 'crop_nocrop_medcrop' | 'crop_nocrop_medcrop_tree' | 'veg' | 'cropType' or 'single_X' (where X is any unique string in the USE_NAME column) for binary classification of X vs all else) 

In [ ]:
rf0 = rf_model(classification_params['pixdf'],
         classification_params['model_dir'],
         classification_params['lc_mod'],
         classification_params['impmeth'],
         classification_params['ranhold'],
         classification_params['model_name'],
         lut)

## view confusion matrices
Note parameters: (pred_col, obs_col, lut, lc_mod_map, lc_mod_acc, print_cm=False, out_dir=None, model_name=None)
To print cm to csv file, change print_cm to True and provide an out_dir and model_Name

In [ ]:
out_dir = os.path.join(classification_params['local_dir']/'cms')

cm_cropNoCrop = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'cropNoCrop',True, os.path.join(out_dir,'cropNoCrop_cms'),classification_params['model_name'])
cm_cropType = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'cropType',True,os.path.join(out_dir,'cropType_cms'),classification_params['model_name'])
cm_veg = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'veg',True,os.path.join(out_dir,'veg_cms'),classification_params['model_name'])
cm_all = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],'all',True,os.path.join(out_dir,'all_cms'),classification_params['model_name'])
#cm_single = get_confusion_matrix(rf0[1]['pred'],rf0[1]['label'],lut,classification_params['lc_mod'],classification_params['lc_mod'],False,classification_params['local_model_dir'],None)

print(cm_cropNoCrop)
#print(cm_cropType)
#print(cm_veg)
#print(cm_all)
#print(cm_single)

print("Done!")

## view variable importance
this can be computed via Impurity or Permutation method (see sklearn docs)  by setting impmeth in rf_model
The full list is stored in the model directory for further manipulation 

In [ ]:
var_imp_path = os.path.join(classification_params['model_dir'],'VarImportance_{}.csv'.format(classification_params['model_name']))
var_imp = pd.read_csv(var_imp_path, names=['var','imp'], header=None)
## view 10 most important variables:
var_imp.sort_values('imp', ascending=False).head(10)

In [ ]:
def get_smallholder(rf_mod, one = False, half = False):
    rf_mod = pd.DataFrame(rf_mod)
    if one == True: #decide if you want to do 1ha or halfha
        smalls = rf_mod.loc[rf_mod['smalls_1ha'] == 1]
        print('There are {} 1ha smallholder samples in the holdout.'.format(smalls.shape[0]))
    elif half == True: 
        smalls = rf_mod.loc[rf_mod['smalls_halfha'] == 1]
        print('There are {} half-ha smallholder samples in the holdout.'.format(smalls.shape[0]))
    
    return smalls

def small_acc(rf_mod, one = False, half = False):
    if one == True:
        one_ha = get_smallholder(rf_mod[1], one = True) # df for 1_ha smallholder ag
        one_ha_cropNoCrop = get_confusion_matrix(one_ha['pred'], one_ha['label'],lut,classification_params['lc_mod'],'cropNoCrop',False,classification_params['model_dir'],None)
        return one_ha_cropNoCrop.head(1)["crop"]/one_ha_cropNoCrop.head(1)["All"]
    
    if half == True:
        half_ha = get_smallholder(rf_mod[1], half = True) # df for half_ha smallholder ag
        half_ha_cropNoCrop = get_confusion_matrix(half_ha['pred'], half_ha['label'],lut,classification_params['lc_mod'],'cropNoCrop',False,classification_params['model_dir'],None)
        return half_ha_cropNoCrop.head(1)["crop"]/half_ha_cropNoCrop.head(1)["All"]


In [ ]:
## take the cms and find take the averages of the UAs and PAs
'''
# only run this section once to create the csvs and then comment out
feature_model = classification_params['feature_model']
sample_model = classification_params['sample_model']
model_name = classification_params['model_name']
out_dir = os.path.join(classification_params['local_dir'],'cms')

cropNoCrop_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'cropNoCrop_cms','{}_cropNoCrop.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'cropNoCrop_cms','{}_cropNoCrop.csv'.format(model_name)), pacc = True)],
                             "1_ha": [small_acc(rf0, one = True)[0]], #create a new function that uses the smallholder rf outputs in a useful manner
                             "half_ha": [small_acc(rf0, half = True)[0]],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
cropType_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(model_name)), pacc = True)],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
veg_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'veg_cms','{}_veg.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'veg_cms','{}_veg.csv'.format(model_name)), pacc = True)],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
all_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [wave(os.path.join(out_dir,'all_cms','{}_all.csv'.format(model_name)), uacc = True)],
                             "PA": [wave(os.path.join(out_dir,'all_cms','{}_all.csv'.format(model_name)), pacc = True)],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})

print(cropNoCrop_metrics)
print(cropType_metrics)
print(veg_metrics)
print(all_metrics)

types = ["cropNoCrop", "cropType", "veg", "all"] 
matrics = [cropNoCrop_metrics, cropType_metrics, veg_metrics, all_metrics]

#saves the cms to the /testing/metrics folder
for idx, i in enumerate(types):
    matrics[idx].to_csv(os.path.join(out_dir,'metrics','{}_metrics.csv'.format(i)))
'''

In [ ]:
# open the csvs and add a new row with the new data

def new_wave(cm_path, stored_path, model_name):
    stored = pd.read_csv(stored_path, index_col = 0)
    new_data = pd.DataFrame({"Model": [model_name],
                             "UA": [wave(cm_path, uacc = True)],
                             "PA": [wave(cm_path, pacc = True)],
                             "1_ha": [small_acc(rf0, one = True)[0]],
                             "half_ha": [small_acc(rf0, half = True)[0]],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
    #print(stored.reset_index(drop=True))
    #print(new_data.reset_index(drop=True))
    stored = pd.concat([stored.reset_index(drop = True), new_data.reset_index(drop = True)], ignore_index = True)
    return stored


In [ ]:
out_dir = os.path.join(classification_params['local_dir'],'cms')
types = ["cropNoCrop", "cropType", "veg", "all"]
mats = [cm_cropNoCrop, cm_cropType, cm_veg, cm_all]

#updates the metric csvs in /testing/metrics folder
for idx, i in enumerate(types):
    mats[idx] = new_wave(os.path.join(out_dir,'{}_cms'.format(i),'{}_{}.csv'.format(classification_params['model_name'], i)), 
                os.path.join(out_dir,'metrics','{}_metrics.csv'.format(i)),classification_params['model_name'])
    mats[idx].to_csv(os.path.join(out_dir,'metrics','{}_metrics.csv'.format(i)))
    print(mats[idx])


In [ ]:
## use this to remove certain rows from the dfs that you do not want anymore

"""
types = ["cropNoCrop", "cropType", "veg", "all"]

delrow = 0
for i in types:
    mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i), index_col = 0)
    
    mat = mat.drop([delrow])
    print(mat)
    mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/{}_metrics.csv".format(i))
"""


In [ ]:
def overall_wave(cnc_metrics, ct_path, model_name):
    
    # iterate through rows of the CNC_metrics dataframe
    cnc = pd.read_csv(cnc_metrics, index_col = 0)
    cnc_partial = cnc[cnc["Model"] == model_name]

    # now deal with the CT matrix
    ct = pd.read_csv(ct_path, index_col = 0)
    ct_partial = [wave(ct_path, uacc = True, weights = "CT"), wave(ct_path, pacc = True, weights = "CT")]
    
    cnc_partial = [cnc_partial["UA"], cnc_partial["PA"], cnc_partial["1_ha"], cnc_partial["half_ha"]]
    #print(cnc_partial)
    #print(ct_partial)

    overall_metrics = pd.DataFrame({"Model": ["{}".format(model_name)],
                             "UA": [(2 * cnc_partial[0] + ct_partial[0])/3],
                             "PA": [(2 * cnc_partial[1] + ct_partial[1])/3],
                             "1_ha": [cnc_partial[2]],
                             "half_ha": [cnc_partial[3]],
                             "No. of Obs.": [len(classification_params["pixdf"]["LC25_name"])]})
    return overall_metrics


In [ ]:
out_dir = os.path.join(classification_params['local_dir'],'cms')
print(overall_wave(os.path.join(out_dir,'metrics','cropNoCrop_metrics.csv'), 
                        os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(classification_params['model_name'])), 
                        classification_params['model_name']))

In [ ]:
## create the baseline for the new overall metrics, once again comment this code out after running it for the first time
'''
overall = overall_wave(os.path.join(out_dir,'metrics','cropNoCrop_metrics.csv'), 
             os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(classification_params['model_name'])),
                       classification_params['model_name'])
overall.to_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'))
'''

In [ ]:
## add new rows to the overall_metrics csv

stored = pd.read_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'), index_col = 0)
new_data = overall_wave(os.path.join(out_dir,'metrics','cropNoCrop_metrics.csv'), 
                        os.path.join(out_dir,'cropType_cms','{}_cropType.csv'.format(classification_params['model_name'])), 
                       classification_params['model_name'])
#print(stored.reset_index(drop=True))
#print(new_data.reset_index(drop=True))
stored = pd.concat([stored.reset_index(drop = True), new_data.reset_index(drop = True)], ignore_index = True)
print(stored)
stored.to_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'))

In [ ]:
## use this to remove certain rows from the df that you do not want anymore

"""
delrow = 16
mat = pd.read_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv", index_col = 0)
mat = mat.drop([delrow])
print(mat)
mat.to_csv("/home/ryanashraf/LUCinSA_helpers/testing/metrics/overall_metrics.csv")
"""


## Make some tables that organize the data cleanly

In [ ]:
## Organize all of the models that exclude poly_pred variables

key_word = "Max_no_pp_"
model_name = classification_params['model_name']

stored = pd.read_csv(os.path.join(out_dir,'metrics','overall_metrics.csv'), index_col = 0)
tab = []
for i in stored.iterrows():
    if i[1]["Model"] == "Max_{}".format(model_name):
        tab.append(i[1])
    if i[1]["Model"].startswith(key_word):
        tab.append(i[1])

tab = pd.DataFrame(tab)
print(tab)


In [ ]:
## 




In [ ]:
## some cool graphs of the data you could have compiled








## To save an html copy of this notebook with all outputs:
(these two cells should be last in notebook)

In [ ]:
%%script echo skipping
### comment out above line and run this cell to print output as html

out_name = str(basic_config['country']+'6c_RandomFoest_ModelComparisons'+'_model'+str(classification_params['model_name'])+'basic_config['filter_yr'])
!jupyter nbconvert --output-dir='./Outputs' --to html --no-input --ExecutePreprocessor.store_widget_state=True --output=$out_name 6b_RandomFoest_ModelComparisons.ipynb